In [1]:
#Tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

#Pandas and numpy for data formats
import pandas as pd
import numpy as np


#glob for data import
import glob
import random
#PyTorch
import torch
import torch.nn as nn
#SK Learn
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
#from sklearn.utils import shuffle
import math
import sys
#MatPlotLib
import matplotlib.pyplot as plt

from models import simpleRegression
from models import feedForward, sequentialModel, sequentialDropout, lstmModel, cnnModel, cnnModelFilters, scoringSystem

#import functions
from dataImport import importTrainingData, importParamsData, importTestData, importTrainParameter, importTargetParameter, importEvalParameter
from preprocessing import preprocessData, createTrainTensors, createParamsTensors, preprocessParams, rescaleData, preprocessParameters


#importTrainData()
train_data = np.load("../train_40000.npy")
params_data = np.load("../train_target_40000.npy")
train_parameter = np.load("../train_parameter_40000.npy")
#
validation_data = np.load("../val_train_40000.npy")
validation_params = np.load("../val_target_40000.npy")
validation_parameter = np.load("../val_parameter_40000.npy")

#Create stats for zScore
mean_value_target = np.mean(params_data)
standard_deviation_target = np.std(params_data)

mean_value = np.mean(train_data)
standard_deviation = np.std(train_data)
measurements_to_remove = 30


median_train = False
replaceValuesBiggerOne_train=False
replaceFirstXValues_train=False
removeFirstXValues_train=True
normalizeData_train=False
zScore_train=True

train_data = preprocessData(train_data,
                          median=median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

train_target = preprocessData(params_data,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)

validation_data = preprocessData(validation_data,
                          median = median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

validation_target = preprocessData(validation_params,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)


number_of_measurements = 300
if removeFirstXValues_train == True:
    number_of_measurements = 300 - measurements_to_remove    
    
#reshaping train data
if type(train_data) != np.ndarray:
    train_data = train_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:
    train_data = train_data.reshape(-1, 55, number_of_measurements)

#reshaping target data
if type(params_data) != np.ndarray:
    params_data = params_data.to_numpy()

#reshaping val data
if type(validation_data) != np.ndarray:
    validation_data = validation_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:    
    validation_data = validation_data.reshape(-1, 55, number_of_measurements)

#reshaping val target
if type(validation_params) != np.ndarray:
    validation_params = validation_params.to_numpy()

    
def scaleToMaxToOne(dat, min, max):
    scaled_dat = dat / max
    return scaled_dat

def unscaleMaxToOne(scaled_dat, min, max):
    unscaled_dat = scaled_dat * max
    return unscaled_dat

min_target = np.min(train_target) 
max_target = np.max(train_target)
print(f"Rescaling targets with max = {max_target}")
train_target = scaleToMaxToOne(train_target, min_target, max_target)
#val_target = scaleToMaxToOne(validation_target, min_target, max_target)    
val_target = validation_target    


prevent_overfitting = keras.callbacks.EarlyStopping(monitor="val_loss", patience = 20, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_delta=0.0001)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Rescaling targets with max = 0.6713607926990712


In [2]:
train_parameter = train_parameter.reshape(-1, 6)
validation_parameter = validation_parameter.reshape(-1, 6)

train_data_reshaped = train_data.reshape(40000, 1, 55, 270)
val_data_reshaped = validation_data.reshape(10000, 1, 55, 270)

In [3]:
def create_concated_model(av = 'selu'):
    first_input = Input(shape=(1, 55, number_of_measurements))

    second_input = Input(shape=(6,))
    
    first_conv = Conv2D(filters=32, kernel_size=(5, 5), data_format='channels_first')(first_input)
    first_pooling = MaxPooling2D(pool_size = 2)(first_conv)
    second_conv = Conv2D(filters=32, kernel_size=(5, 5), data_format='channels_first')(first_pooling)
    second_pooling = MaxPooling2D(pool_size = 2)(second_conv)
    third_conv = Conv2D(filters=32, kernel_size=(5, 5), data_format='channels_first')(second_pooling)
    third_pooling = MaxPooling2D(pool_size = 2)(third_conv)
    
    first_dense = Dense(1024, activation=av)(third_pooling)
    second_dense = Dense(1024, activation=av)(first_dense)
    third_dense = Dense(1024, activation=av)(second_dense)
    fourth_dense = Dense(1024, activation=av)(third_dense)
    fifth_dense = Dense(1024, activation=av)(fourth_dense)
    sixth_dense = Dense(1024, activation=av)(fifth_dense)
    flatten_layer = tf.keras.layers.Flatten()(sixth_dense)


    parameter_first_dense = Dense(10, activation=av)(second_input)
    parameter_second_dense = Dense(10, activation=av)(parameter_first_dense)
    parameter_third_dense = Dense(10, activation=av)(parameter_second_dense)
    parameter_fourth_dense = Dense(10, activation=av)(parameter_third_dense)
    
    concatLayer = tf.keras.layers.concatenate([parameter_fourth_dense, flatten_layer]) 


    output = Dense(55)(concatLayer)


    model = Model(inputs=[first_input, second_input], outputs=output)
    optim = keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(optimizer=optim,
                  loss='mae')
        
    return model

model = create_concated_model()
model2 = create_concated_model()
model3 = create_concated_model()



# Model 1

In [ ]:
history_test_1 = model.fit((train_data_reshaped, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 19s 16ms/step - loss: 0.6398 - val_loss: 0.0443
Epoch 2/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0378 - val_loss: 0.0353
Epoch 3/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0332 - val_loss: 0.0396
Epoch 4/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0328 - val_loss: 0.0358
Epoch 5/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0309 - val_loss: 0.0291
Epoch 6/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0297 - val_loss: 0.0285
Epoch 7/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0305 - val_loss: 0.0298
Epoch 8/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0272 - val_loss: 0.0292
Epoch 9/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0270 - val_loss: 0.0278
Epoch 10/1000
1000/1000 [=====================

1000/1000 [==============================] - 15s 15ms/step - loss: 0.0021 - val_loss: 0.0067
Epoch 78/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0021 - val_loss: 0.0060
Epoch 79/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0020 - val_loss: 0.0065
Epoch 80/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0021 - val_loss: 0.0060
Epoch 81/1000
1000/1000 [==============================] - 15s 15ms/step - loss: 0.0020 - val_loss: 0.0065
Epoch 82/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0020 - val_loss: 0.0061
Epoch 83/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0019 - val_loss: 0.0064
Epoch 84/1000
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0018 - val_loss: 0.0065
Epoch 85/1000
1000/1000 [==============================] - 16s 16ms/step - loss: 0.0018 - val_loss: 0.0061
Epoch 86/1000
1000/1000 [==========================

In [ ]:
pred = model.predict((val_data_reshaped, validation_parameters))
pred_final = unscaleMaxToOne(pred, min_target, max_target)
score = scoringSystem(validation_target, pred_final)

print(score)

In [ ]:
last_five_idx = len(history_test_1.history["val_loss"])-5
val_loss_1 = sum(history_test_1.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1.history["loss"])-5
train_loss_1 = sum(history_test_1.history["loss"][last_five_idx:])/5

# Model 2

In [ ]:
history_test_2 = model2.fit((train_data_reshaped, train_parameter),
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

In [ ]:
pred2 = model2.predict(val_data_reshaped)
pred_final2 = unscaleMaxToOne(pred2, min_target, max_target)
score2 = scoringSystem(validation_target, pred_final2)

print(score2)

In [ ]:
last_five_idx = len(history_test_2.history["val_loss"])-5
val_loss_2 = sum(history_test_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_2.history["loss"])-5
train_loss_2 = sum(history_test_2.history["loss"][last_five_idx:])/5

# Model 3

In [ ]:
history_test_3 = model3.fit(train_data_reshaped,
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

In [ ]:
pred3 = model3.predict(val_data_reshaped)
pred_final3 = unscaleMaxToOne(pred3, min_target, max_target)
score3 = scoringSystem(validation_target, pred_final3)

print(score3)

In [ ]:
last_five_idx = len(history_test_3.history["val_loss"])-5
val_loss_3 = sum(history_test_3.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_3.history["loss"])-5
train_loss_3 = sum(history_test_3.history["loss"][last_five_idx:])/5

# Evaluation

In [ ]:
val_loss_final = (val_loss_1 + val_loss_2 + val_loss_3)/3
train_loss_final = (train_loss_1 + train_loss_2 + train_loss_3)/3
score_final = (score + score2 + score3)/3

In [ ]:
print("Final train loss: ", train_loss_final)
print("Final validation loss: ", val_loss_final)
print("Final Score: ", score_final)